# Kernel Tuner Simulation Mode Test

This notebook is intended to examine the completeness, consistency, and advantage of simulation mode in Kernel Tuner.

In [1]:
from pathlib import Path
import json
import re
import pandas as pd
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
from warnings import warn
from typing import Tuple

cachedirectory = Path("kernels/outputdata/simulation_mode")
savedirectory = Path(Path.cwd(), "figures", "simulation_mode")
mpl.rcParams["savefig.directory"] = savedirectory

In [2]:
# arguments
kernelname = "gemm_cltune_opencl"
expected_keys_env = ['execution_time', 'overhead_time', 'total_benchmark_time', 'total_verification_time', 'total_compile_time', 'total_strategy_time', 'total_framework_time']
expected_keys_cache = ['time', 'times', 'GFLOP/s', 'temperature', 'compile_time', 'benchmark_time', 'verification_time', 'framework_time', 'strategy_time']

algorithm_displaynames = {
    'brute': "bruteforce",
    'random': "random",
}

plot_height = 6
plot_aspect = 2.0
plot_dpi = 250

In [3]:
def get_dataframes() -> Tuple[pd.DataFrame, pd.DataFrame]:

    # collect the relevant cachefiles
    cachefiles : list[Path] = []
    for file in cachedirectory.iterdir():
        if file.is_file() and file.name.startswith(kernelname) and file.name.endswith('_output.json'):
            cachefiles.append(file)

    # create a list of dictionaries of the data
    cache_dicts = list()
    env_dicts = list()
    for filepath in cachefiles:
        # load the data
        envfilepath = Path(str(filepath).replace('_output', '_env'))
        assert envfilepath.exists()
        try:
            cachefile = json.load(filepath.open())
            envfile = json.load(envfilepath.open())
        except json.JSONDecodeError:
            warn(f"Could not load cachefile {filepath.name}")
            continue
        assert isinstance(cachefile, list)
        assert isinstance(envfile, dict)

        # combine the data into a dictionary
        device_name = envfile['device_name']
        is_simulated = "_mode=simulated" in filepath.name
        algorithm = algorithm_displaynames[re.search(r"_alg=([a-zA-Z]+)_", filepath.name).group(1).strip()]
        # for each of the tuned configurations, create a dictionary with the expected items
        for value in cachefile:
            assert isinstance(value, dict)
            cache = {}
            cache['device_name'] = device_name
            cache['algorithm'] = algorithm
            cache['simulated'] = is_simulated
            for key in expected_keys_cache:
                cache[key] = value[key]
            cache['total_time'] = value['strategy_time'] + value['compile_time'] + value['benchmark_time'] + value['verification_time'] + value['framework_time']
            cache_dicts.append(cache)
        # create a global dictionary per tuning session
        env = {}
        env['device_name'] = device_name
        env['algorithm'] = algorithm
        env['simulated'] = is_simulated
        for key in expected_keys_env:
            env[key] = envfile[key]
        env_dicts.append(env)

    # create a dataframe out of the dictionaries
    df_cache = pd.DataFrame(cache_dicts)
    df_env = pd.DataFrame(env_dicts)
    return df_cache, df_env

In [6]:
df_cache, df_env = get_dataframes()
df_env

,device_name,algorithm,simulated,execution_time,overhead_time,total_benchmark_time,total_verification_time,total_compile_time,total_strategy_time,total_framework_time
0,NVIDIA RTX A4000,random,False,126.947397,5023.528329,111761.298259,0,9749.827428,337.611018,64.474528
1,NVIDIA RTX A4000,random,True,9.154694,546.212684,112379.759576,0,8773.168948,7989.141526,1.568631
2,NVIDIA RTX A4000,random,True,9.913607,510.762541,110466.556431,0,9567.722916,8813.888148,1.626062
3,NVIDIA RTX A4000,random,False,123.006391,4972.587217,106228.356257,0,11387.631305,337.772861,66.008131
4,NVIDIA RTX A4000,random,False,122.914690,6714.116277,105004.329495,0,10794.750350,311.515601,78.215190
5,NVIDIA RTX A4000,random,False,124.372556,4688.260592,107122.306403,0,12143.325786,338.603897,69.660249
6,NVIDIA RTX A4000,random,True,9.503639,522.781810,119946.835716,0,9198.048295,8391.983658,1.625797
7,NVIDIA RTX A4000,random,True,8.708234,488.979616,115367.033540,0,8335.427660,7631.148135,1.595390
8,NVIDIA RTX A4000,random,False,121.956704,4705.043546,104818.101858,0,12013.035724,341.195311,67.268876
9,NVIDIA RTX A4000,random,True,9.728281,516.639284,113522.744807,0,8943.893163,8625.353546,1.548823
